# 

In [30]:
import pandas as pd
import sourmash
import glob
import os

In [6]:
fmg_csv = "../output.roux2017/fastmultigather/roux2017-x-refseq-v219.dna.k21-sc10.t0.gather.csv"

In [8]:
samplecomp_tsvs = glob.glob('../roux2017-samplecomp/*tsv')
samplecomp_tsvs

['../roux2017-samplecomp/Sample_10_comp.tsv',
 '../roux2017-samplecomp/Sample_11_comp.tsv',
 '../roux2017-samplecomp/Sample_7_comp.tsv',
 '../roux2017-samplecomp/Sample_6_comp.tsv',
 '../roux2017-samplecomp/Sample_1_comp.tsv',
 '../roux2017-samplecomp/Sample_4_comp.tsv',
 '../roux2017-samplecomp/Sample_5_comp.tsv',
 '../roux2017-samplecomp/Sample_13_comp.tsv',
 '../roux2017-samplecomp/Sample_12_comp.tsv',
 '../roux2017-samplecomp/Sample_3_comp.tsv',
 '../roux2017-samplecomp/Sample_2_comp.tsv',
 '../roux2017-samplecomp/Sample_14_comp.tsv',
 '../roux2017-samplecomp/Sample_9_comp.tsv',
 '../roux2017-samplecomp/Sample_8_comp.tsv']

In [33]:
cmpDF = pd.concat((pd.read_csv(f, sep='\t').assign(Sample=os.path.basename(f).split('_comp.tsv')[0]) for f in samplecomp_tsvs), ignore_index=True)
cmpDF.rename(columns={'## Virus': 'Virus'}, inplace=True)
cmpDF.head()

,Virus,Name,Coverage,Sample
0,22855216,Pseudomonas phage phi-12 segment S,0.058544,Sample_10
1,134288688,Burkholderia phage phiE12-2 chromosome,0.054693,Sample_10
2,530787156,Bacillus phage Troll,0.051090,Sample_10
3,269838908,Morganella phage MmP1,0.047719,Sample_10
4,557307743,Mycobacterium phage Fredward,0.044565,Sample_10


In [48]:
fmg = pd.read_csv(fmg_csv)
fmg['Name'] = fmg['match_name'].str.split(' ', n=1, expand=True)[1]
fmg.head()

,query_name,query_md5,match_name,match_md5,f_match_query,intersect_bp,Name
0,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|388684650|ref|NC_017972.1| Pseudomonas phag...,8e4254b5af76d576145006347d533040,0.999964,281400,"Pseudomonas phage Lu11, complete genome"
1,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|414088861|ref|NC_019408.1| Caulobacter phag...,e1376543b913375166cbb1750b40e2e4,1.000000,213260,"Caulobacter phage CcrRogue, complete genome"
2,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|418487144|ref|NC_019443.1| Synechococcus ph...,f9f22f4efe6f9e4aa35902ba4dd5b534,0.999710,172550,"Synechococcus phage metaG-MbCM1, complete genome"
3,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|460042674|ref|NC_020486.1| Synechococcus ph...,f5d4f2e624ffb0caed5a7abe11c536df,0.995892,170300,"Synechococcus phage S-RIM8 A.HR1, complete genome"
4,Sample_5,cc604cf6c7bbd9634d06112a8bcacdb3,gi|29366675|ref|NC_000866.4| Enterobacteria ph...,f7691cb10a71235d9ef18cfab0b9a642,0.999701,167200,"Enterobacteria phage T4, complete genome"


In [ ]:
fmg.groupby(['query_name', 'match_name'])